In [ ]:
import pandas as pd
import numpy as np
import requests, json 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from time import sleep
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tensorflow
import numpy as np
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, TimeDistributed
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
pd.options.mode.chained_assignment = None

### LSTM Model with Multi-Output

In [ ]:
################################################
# Sample data with multiple inputs and outputs #
################################################

def sample_time_series_roll(N, T, seed = 1337):
    np.random.seed(seed)
    
    # Inputs
    x1 = np.array(np.random.uniform(size=2*T*N)).reshape(2*N, T)
    x2 = np.array(np.random.uniform(size=2*T*N)).reshape(2*N, T)
    x3 = np.array(np.random.uniform(size=2*T*N)).reshape(2*N, T)
    x4 = np.array(np.random.uniform(size=2*T*N)).reshape(2*N, T)
    
    # Outputs
    y1 = np.roll(x1, 2) # y1[t] = x1[t-2]
    y2 = np.roll(x2, 1) * np.roll(x3, 2) # y2[t] = x2[t-1]*x3[t-2]
    y3 = np.roll(x4, 3) # y3[t] = x4[t-3]
   
   
    # Training/test sets
    x1_train, x2_train, x3_train, x4_train = [x[0:N] for x in [x1, x2, x3, x4]]
    x1_test, x2_test, x3_test, x4_test = [x[N:2*N] for x in [x1, x2, x3, x4]]
    y1_train, y2_train, y3_train = [y[0:N] for y in [y1, y2, y3]]
    y1_test, y2_test, y3_test = [y[N:2*N] for y in [y1, y2, y3]]
    
    return(x1_train, x2_train, x3_train, x4_train, x1_test, x2_test, x3_test, 
           x4_test,y1_train, y2_train, y3_train,y1_test, y2_test, y3_test)

def sample_time_series_1(N, T, seed = 1337):
    np.random.seed(seed)
    ##
    # Inputs
    ##
    x = np.zeros(2*N*T).reshape(2*N, T)
    one_indexes = np.random.choice(a = 2*N, size = N, replace = False)
    x[one_indexes, 0] = 1 # very long term memory
    ##
    # Outputs
    ##
    y = np.repeat(x[:,0], T).reshape(2*N, T)
    ##
    # Training/test sets
    ##
    x_train = x[0:N]
    x_test = x[N:2*N]
    y_train = y[0:N]
    y_test = y[N:2*N]
    return(x_train, x_test, y_train, y_test)


# Plotting loss and val_loss as function of epochs 
def plotting(history):
    plt.plot(history.history['loss'], color = "red")
    plt.plot(history.history['val_loss'], color = "blue")
    red_patch = mpatches.Patch(color='red', label='Training')
    blue_patch = mpatches.Patch(color='blue', label='Test')
    plt.legend(handles=[red_patch, blue_patch])
    plt.xlabel('Epochs')
    plt.ylabel('MSE loss')
    plt.show()

### Helper functions for stateful network (i.e. for Part C) 

In [ ]:
def stateful_cut(arr, batch_size, T_after_cut):
    if len(arr.shape) != 3:
        # N: Independent sample size,
        # T: Time length,
        # m: Dimension
        print("ERROR: please format arr as a (N, T, m) array.")
    
    N = arr.shape[0]
    T = arr.shape[1]
    # We need T_after_cut * nb_cuts = T
    nb_cuts = int(T / T_after_cut)
    if nb_cuts * T_after_cut != T:
        print("ERROR: T_after_cut must divide T")
    
    # We need batch_size * nb_reset = N
    nb_reset = int(N / batch_size)
    if nb_reset * batch_size != N:
        print("ERROR: batch_size must divide N")
   
    # Cutting (technical)
    cut1 = np.split(arr, nb_reset, axis=0)
    cut2 = [np.split(x, nb_cuts, axis=1) for x in cut1]
    cut3 = [np.concatenate(x) for x in cut2]
    cut4 = np.concatenate(cut3)
    
    return(cut4)

In [ ]:
##############################################################
# Function to define 'Callback resetting model states' class #
##############################################################
# This callback will slow down computations.
def define_reset_states_class(nb_cuts):
    class ResetStatesCallback(Callback):
        def __init__(self):
            self.counter = 0
        def on_batch_begin(self, batch, logs={}):
            # We reset states when nb_cuts batches are completed, as
            # shown in the after cut figure
            if self.counter % nb_cuts == 0:
                self.model.reset_states()
                self.counter += 1
        def on_epoch_end(self, epoch, logs={}):
            # reset states after each epoch
            self.model.reset_states()
    return(ResetStatesCallback)

In [ ]:
def batched(i, arr, batch_size):
    return(arr[i*batch_size:(i+1)*batch_size])

def test_on_batch_stateful(model, inputs, outputs, batch_size, nb_cuts):
    nb_batches = int(len(inputs)/batch_size)
    sum_pred = 0
    for i in range(nb_batches):
        if i % nb_cuts == 0:
            model.reset_states()
            x = batched(i, inputs, batch_size)
            y = batched(i, outputs, batch_size)
            sum_pred += model.test_on_batch(x, y)
            mean_pred = sum_pred / nb_batches
    return(mean_pred)

def define_stateful_val_loss_class(inputs, outputs, batch_size, nb_cuts):
    class ValidationCallback(Callback):
        def __init__(self):
            self.val_loss = []
        def on_epoch_end(self, epoch, logs={}):
            mean_pred = test_on_batch_stateful(self.model, inputs, outputs, 
            batch_size, nb_cuts)
            print('val_loss: {:0.3e}'.format(mean_pred), end = '')
            self.val_loss += [mean_pred]
        def get_val_loss(self):
            return(self.val_loss)
    return(ValidationCallback)

### Preparing Data

In [ ]:
train_ocean = pd.read_csv('Fish_Training_Data.csv')

In [ ]:
X = train_ocean.dropna(subset=['Lat_7_days','Lon_7_days','CPUE_7_days']).sort_values('Date').drop(['CPUE_number_per_hour','Date'],axis=1)#[train_ocean['CPUE']<300] #
y = train_ocean.dropna(subset=['Lat_7_days','Lon_7_days','CPUE_7_days']).sort_values('Date')['CPUE_number_per_hour'] #[train_ocean['CPUE']<300]
X.shape, y.shape

In [ ]:
train_ocean.head(1)

In [ ]:
dataset = train_ocean.dropna(subset=['Lat_7_days','Lon_7_days','CPUE_7_days']).sort_values('Date')
dataset.shape

In [ ]:
N = 16
T = 120
batch_size = 8 # number of time series considered together: batch_size | N
T_after_cut = 15
round_end = 3840
# Inputs
x1_f = dataset['Lat_7_days'].values[:round_end].reshape(2*N, T)
x2_f = dataset['CPUE_7_days'].values[:round_end].reshape(2*N, T)
x3_f = dataset['Lon_7_days'].values[:round_end].reshape(2*N, T)
x4_f = dataset['thetao'].values[:round_end].reshape(2*N, T)
x5_f = dataset['so'].values[:round_end].reshape(2*N, T)

# Outputs
y1_f = dataset['CPUE_number_per_hour'].values[:round_end].reshape(2*N, T)
y2_f = dataset['Lat'].values[:round_end].reshape(2*N, T)
y3_f = dataset['Lon'].values[:round_end].reshape(2*N, T)
x1_f.shape == y1_f.shape

In [ ]:
# Training/test sets
x1_train_f, x2_train_f, x3_train_f, x4_train_f = [x[0:N] for x in [x1_f, x2_f, x3_f, x4_f]]
x1_test_f, x2_test_f, x3_test_f, x4_test_f = [x[N:2*N] for x in [x1_f, x2_f, x3_f, x4_f]]

y1_train_f, y2_train_f, y3_train_f = [y[0:N] for y in [y1_f, y2_f, y3_f]] #! Delete last [0] brackets if more than one output
y1_test_f, y2_test_f, y3_test_f = [y[N:2*N] for y in [y1_f, y2_f, y3_f]] #! Delete last [0] brackets if more than one output
x1_train_f.shape, x1_test_f.shape, y1_train_f.shape, y1_test_f.shape

In [ ]:
# Reshape each time series as (N, T, dim_in) or (N, T, dim_out)
x_train_f = np.concatenate((x1_train_f.reshape(N, T, 1), 
                            x2_train_f.reshape(N, T, 1),
                            x3_train_f.reshape(N, T, 1),
                            x4_train_f.reshape(N, T, 1),
                            #x5_train_f.reshape(N, T, 1),
                           ), axis=2)

y_train_f = np.concatenate((y1_train_f.reshape(N, T, 1),
                            y2_train_f.reshape(N, T, 1),
                            y3_train_f.reshape(N, T, 1),
                            ), axis=2)

x_train_f.shape, y_train_f.shape

In [ ]:
x_test_f = np.concatenate((x1_test_f.reshape(N, T, 1), 
                            x2_test_f.reshape(N, T, 1),
                            x3_test_f.reshape(N, T, 1),
                            x4_test_f.reshape(N, T, 1),
                            #x5_test_f.reshape(N, T, 1),
                          ), axis=2)

y_test_f = np.concatenate((y1_test_f.reshape(N, T, 1),
                            y2_test_f.reshape(N, T, 1),
                            y3_test_f.reshape(N, T, 1),
                            ), axis=2)

x_test_f.shape, y_test_f.shape

In [ ]:
inputs, outputs, inputs_test, outputs_test = \
  [stateful_cut(arr, batch_size, T_after_cut) for arr in \
  [x_train_f, y_train_f, x_test_f, y_test_f]]

# Model

In [ ]:
## Parameters
dim_in = 4 # dimension of input time series
dim_out = 3
nb_units = 100
model = Sequential()
model.add(LSTM(batch_input_shape=(batch_size, None, dim_in),
               return_sequences=True, units=nb_units, stateful=True))

model.add(LSTM(30, activation='relu', return_sequences=True))

model.add(TimeDistributed(Dense(activation='linear', units=dim_out)))

model.compile(loss = 'mse', optimizer = 'rmsprop')

##
# Training
##
epochs = 100
nb_reset = int(N / batch_size)
nb_cuts = int(T / T_after_cut)
if nb_reset > 1:
    ResetStatesCallback = define_reset_states_class(nb_cuts)
    ValidationCallback = define_stateful_val_loss_class(inputs_test, 
    outputs_test, 
    batch_size, nb_cuts)
    validation = ValidationCallback()
    history = model.fit(inputs, outputs, epochs = epochs, 
    batch_size = batch_size, shuffle=False,
    callbacks = [ResetStatesCallback(), validation])
    history.history['val_loss'] = ValidationCallback.get_val_loss(validation)
else:
    # If nb_reset = 1, we should reset states after each epoch.
    # To improve computational speed, we can decide not to reinitialize states
    # at all. Results are similar in this case.
    # In the following line, states are not reinitialized at all:
    history = model.fit(inputs, outputs, epochs = epochs, 
    batch_size = batch_size, shuffle=False,
    validation_data=(inputs_test, outputs_test))

### Checking model performance

In [ ]:
plotting(history)

### Visualising predicted and actual values

In [ ]:
model_stateless = Sequential()
model_stateless.add(LSTM(input_shape=(None, dim_in),
return_sequences=True, units=nb_units))
model_stateless.add(LSTM(30, activation='relu', return_sequences=True))
model_stateless.add(TimeDistributed(Dense(activation='linear', units=dim_out)))
model_stateless.compile(loss = 'mse', optimizer = 'rmsprop')
model_stateless.set_weights(model.get_weights())

## Prediction of a new set
n = 10 # time series selected (between 0 and N-1)
x = x_test_f[n]
y = y_test_f[n]
y_hat = model_stateless.predict(np.array([x]))[0]
dim = 0 # dim=0 for y1 ; dim=1 for y2 ; dim=2 for y3.
size = 35
plt.plot(range(T)[0:size], y[:,dim][0:size])
plt.plot(range(T)[0:size], y_hat[:,dim][0:size])